# Notebook with some Data Cleaning steps

Data Inspection beforehand with Excel to identify patterns and correct changes, might be quite different from current approach

### Fill missing Values

In [ ]:
import pandas as pd

# Read Files
demanders = pd.read_excel('Busses_Timeseries_Table.xlsx')
generators = pd.read_excel('Generators_Timeseries_Table_.xlsx')
generatorsGrid = pd.read_excel('Generators_Table.xlsx')
branches = pd.read_excel('Branches_Input_Table.xlsx')

temp = pd.Timestamp(generators.loc[0, 'bid_datetime_beginning_utc'])
nGen = len(generators.loc[generators['bid_datetime_beginning_utc'] == temp])

In [ ]:
# Mising data: Empty values (capacity for generator 1)
import random
import math

index = 0
emptyCount = 0
for i in range(len(generators.index)):
    row = generators.iloc[[i]]
    if row['unit_code'].item() == "AAAABg8LDwQAMS45NjYyMjEwNA" and row['max_ecomax'].item() == 0: # Fill empty generator 1 capacity values
        generators.at[i, 'max_ecomax'] = generators.loc[emptyCount * nGen, 'max_ecomax'] # Fill with historic value
        emptyCount += 1
        if emptyCount == 1:
            print("Empty found")
    if row['unit_code'].item() == "AAAABg8LDwQAMS45NjYyMjEwNA" and row['max_ecomax'].item() != 0:
        emptyCount = 0


In [ ]:
# Add missing generator values
missingIDs = ['AAAADQYJAgAFLjEwNTAyMjEy', 'AAAADQYJAgAHLjEwNTAyMjEw', 'AAAADQYJAgMPLjEwNTAyMjEx']

nMissingGen = len(missingIDs) # Number of missing generators
givenGen = nGen - nMissingGen
length = len(generators.index)
doit = False
index = 0
while index < length:  
    temp = pd.Timestamp(generators.loc[index, 'bid_datetime_beginning_utc'])
    nGenTemp = len(generators.loc[generators['bid_datetime_beginning_utc'] == temp])

    if nGenTemp < nGen: # Missing data for generators, need to adjust to pattern (same three missing here)
        index += givenGen - 1
        rowcount = 0
        row = generators.iloc[[index]]
        while rowcount < nMissingGen:
            rowlist = row.values.flatten().tolist()
            rowlist[2] = missingIDs[rowcount] # Update with missing ID
            generators.loc[index + rowcount/10] = rowlist # Create new rows after index with same value
            # Set values to 0 or add threshold or sampled gap to variable values
            rowcount += 1
    index += 1

In [ ]:
# Missing Values: Fill missing edge capacities
limAverage = branches[branches['Lim MVA A'] != 0]['Lim MVA A'].mean()
branches.loc[branches['Lim MVA A'] == 0, 'Lim MVA A'] = limAverage

### Adjust Variance

In [ ]:
# Adjust Variance: Reduce outliers (generator 1) with approximate scaling values, sample too similar bids (generator 4-5)
import random
import math

index = 0
emptyCount = 0
for i in range(len(generators.index)):
    row = generators.iloc[[i]]

    # Reduce outliers (specific generator identified)
    if row['unit_code'].item() == "AAAABg8LDwQAMS45NjYyMjEwNA":
        generators.at[i, 'max_ecomax'] = generators.loc[i, 'max_ecomax'] * 0.25
        generators.at[i, 'min_ecomin'] = generators.loc[i, 'min_ecomin'] * 0.1
        if generators.loc[i, 'max_ecomax'] > 260:  #if still bigger reduce again
            generators.at[i, 'max_ecomax'] = generators.loc[emptyCount * 6, 'max_ecomax'] / 373 * 260

    # Sample too similar/filled in data of generators (two specific generators idetified)
    elif row['unit_code'].item() == "AAAADQYJAgAHLjEwNTAyMjEw" or row['unit_code'].item() == "AAAADQYJAgMPLjEwNTAyMjEx":
        gap = random.uniform(0, 15) / 100
        generators.at[i, 'max_ecomax'] = round(generators.loc[i, 'max_ecomax'] * (1-gap), 2)

### Match Identifiers

In [ ]:
# Match Identifiers of Generators

#to be replaced
idsNew = ['AAAABg8LDwQBMS45NjYyMjEwNA', 'AAAADQYJAgAALjEwNTAyMjE0', 'AAAADQYJAgACLjEwNTAyMjEz', 'AAAADQYJAgAELjEwNTAyMjEy', 'AAAADQYJAgAGLjEwNTAyMjEw', 'AAAADQYJAgMOLjEwNTAyMjEx']
#correct / replaced by
idsOld = ['AAAABg8LDwQAMS45NjYyMjEwNA', 'AAAADQYJAgABLjEwNTAyMjE0', 'AAAADQYJAgADLjEwNTAyMjEz', 'AAAADQYJAgAFLjEwNTAyMjEy', 'AAAADQYJAgAHLjEwNTAyMjEw', 'AAAADQYJAgMPLjEwNTAyMjEx']

def change_id (row):
    if row['unit_code'] in idsNew:
        oldIndex = idsNew.index(row['unit_code'])
        return idsOld[oldIndex]
    return row['unit_code']

generators['unit_code'] = generators.apply (lambda row: change_id(row), axis = 1)